In [1]:
def check_gpu():
    import tensorflow as tf
    print("tf.test.is_built_with_cuda()")
    print(tf.test.is_built_with_cuda())
    
    print()
    print("tf.config.list_physical_devices('GPU')")
    print(tf.config.list_physical_devices('GPU'))

    print()
    print("tf.config.experimental.list_physical_devices('GPU')")
    print(tf.config.experimental.list_physical_devices('GPU'))

#check_gpu()

In [2]:
import pandas as pd

df = pd.read_csv('../chessData.csv', nrows=100000)

def isEvaluationNumeric(myStr):
    try:
        myStr = myStr.lstrip('-')
        x = int(myStr)
        return True
    except:
        return False

# Convert the Evaluation column to integers and remove rows where conversion is not possible
df = df[df['Evaluation'].apply(lambda x: isEvaluationNumeric(x))]
df['Evaluation'] = df['Evaluation'].astype(int)

#print("Avg eval from file:", round(df["Evaluation"].mean()))

In [3]:
def evaluateWithSf16():
    from stockfish import Stockfish
    stockfish = Stockfish(path="/users/ricar/desktop/stockfish/stockfish-windows-x86-64-avx2.exe",
                          depth=5, 
                          parameters={"Threads": 4})

    print("Stockfish", stockfish.get_stockfish_major_version())
    #stockfish.set_fen_position("rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2")
    #print(stockfish.get_evaluation()["value"])

    def getStockfishEval(fen):
        stockfish.set_fen_position(fen)
        return int(stockfish.get_evaluation()["value"])
    df['Evaluation'] = df['FEN'].apply(getStockfishEval)

    print("Avg eval stockfish 16:", round(df["Evaluation"].mean()))
    
#evaluateWithSf16()

In [4]:
import numpy as np
import chess

def bitboard_to_array(bb: int) -> np.ndarray:
    s = 8 * np.arange(7, -1, -1, dtype=np.uint64)
    b = (bb >> s).astype(np.uint8)
    b = np.unpackbits(b, bitorder="little")
    return b.reshape(8, 8)

def bitboards_to_array(bb: np.ndarray) -> np.ndarray:
    bb = np.asarray(bb, dtype=np.uint64)[:, np.newaxis]
    s = 8 * np.arange(7, -1, -1, dtype=np.uint64)
    b = (bb >> s).astype(np.uint8)
    b = np.unpackbits(b, bitorder="little")
    return b.reshape(-1, 8, 8)

def fen_to_bitboards(myFen):
    # outputs (12,8,8)
    board = chess.Board(myFen)
    black, white = board.occupied_co
    bitboards = np.array([
        black & board.pawns,
        black & board.knights,
        black & board.bishops,
        black & board.rooks,
        black & board.queens,
        black & board.kings,
        white & board.pawns,
        white & board.knights,
        white & board.bishops,
        white & board.rooks,
        white & board.queens,
        white & board.kings,
    ], dtype=np.uint64)
    return bitboards_to_array(bitboards)

In [19]:
#import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import backend as K
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Input, concatenate

X_fen = df['FEN'].values  # FEN positions
X_bitboards_list = [fen_to_bitboards(fen) for fen in X_fen]
X_bitboards = np.stack(X_bitboards_list)
y = df['Evaluation'].values
X_train, X_test, y_train, y_test = train_test_split(X_bitboards, y, test_size=0.2, random_state=42)

X_bool_train = np.random.randint(0, 2, size=(X_train.shape[0], 1))
X_bool_test = np.random.randint(0, 2, size=(X_test.shape[0], 1))

In [21]:
conv_model = Sequential()
conv_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(12, 8, 8)))
conv_model.add(Flatten())
conv_model.add(Dense(32, activation='relu'))
conv_model.add(Dense(8, activation='relu'))
bool_input = Input(shape=(1,), dtype='float32')
combined = concatenate([conv_model.output, bool_input])
output_layer = Dense(1)(combined)
model.add(Dense(1))  # Output layer (single value prediction)
model = Model(inputs=[conv_model.input, bool_input], outputs=output_layer)

model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 conv2d_11_input (InputLayer)   [(None, 12, 8, 8)]   0           []                               
                                                                                                  
 conv2d_11 (Conv2D)             (None, 10, 6, 32)    2336        ['conv2d_11_input[0][0]']        
                                                                                                  
 flatten_11 (Flatten)           (None, 1920)         0           ['conv2d_11[0][0]']              
                                                                                                  
 dense_30 (Dense)               (None, 32)           61472       ['flatten_11[0][0]']             
                                                                                              

In [ ]:
def rmse(y_true, y_pred):
        return K.sqrt(K.mean(K.square(K.cast(y_pred, dtype=K.floatx()) - K.cast(y_true, dtype=K.floatx()))))

model.compile(loss=rmse, optimizer=Adam(learning_rate=0.001)) # default lr 0.001
history = model.fit([X_train, X_bool_train], y_train, epochs=10, batch_size=64, validation_data=([X_test, X_bool_test], y_test))

print("Evaluating on test set...")
loss = model.evaluate(X_test, y_test)

print("Evaluating on test set (batch_size=128)...")
loss = model.evaluate(X_test, y_test, batch_size=128)

In [ ]:
import matplotlib.pyplot as plt

# Plot training and validation loss over epochs
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss (RMSE)')
plt.legend()
plt.show()